In [4]:
import numpy as np
from skimage import io,transform
import pandas as pd
from os import path
from glob import glob
from natsort import natsorted
import matplotlib.pyplot as plt

import stackview

In [5]:

experiment_dir = '/Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Dermal topography/01-26-2026 mTmG'
region_dirs = glob(path.join(experiment_dir,'F2*/*R1/'))
region_dir = region_dirs[0]

channel_names = ['R','G','B']

reference_frame = 1

reference_points = natsorted(glob(path.join(region_dir,'Landmarks/t*.csv')))
reference_points = pd.concat([pd.read_csv(f,index_col=0) for f in reference_points],ignore_index=True)
reference_points = reference_points.rename(columns={'axis-0':'Z','axis-1':'Y','axis-2':'X'})
reference_points.index.name = 'Frame'

# Calculate deltas to the reference frame
deltas = reference_points - reference_points.loc[reference_frame].astype(int)
deltas.astype(int)

,Z,Y,X
Frame,,,
0,-13,12,-143
1,0,0,0
2,5,0,-87
3,-5,-85,-305
4,4,4,-176


In [6]:
# Load images
day_directories = natsorted(glob(path.join(region_dir,'*. Day*')))
TT = len(day_directories)

channels = {}
for c in channel_names:
    filenames = natsorted(glob(path.join(region_dir,f'*. Day */{c}_reg.tif')))
    channels[c] = [io.imread(f) for f in filenames]


In [7]:
from skimage import util
from twophotonUtils import z_align_ragged_timecourse

for c in channel_names:
    this_chan = channels[c]
    transformed = []
    for t in range(TT):
        if t == reference_frame:
            transformed.append(util.img_as_float(this_chan[t]))
        else:
            this_chan_this_frame = this_chan[t]
            T = transform.EuclideanTransform(translation=deltas[['X','Y']].loc[t])
            this_chan_this_frame_transformed = np.zeros_like(this_chan_this_frame,dtype=float)
            for z,im in enumerate(this_chan_this_frame):
                this_chan_this_frame_transformed[z,...] = transform.warp(im,T)
            transformed.append(this_chan_this_frame_transformed)
        
    transformed = z_align_ragged_timecourse(transformed,reference_points['Z'])
    io.imsave(path.join(region_dir,f'{c}_align.tif'), transformed)
    

100%|█████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 21.65it/s]
/opt/miniconda3/envs/jupyter/lib/python3.12/site-packages/skimage/_shared/utils.py:328: UserWarning: /Users/xies/Library/CloudStorage/OneDrive-Stanford/Skin/Two photon/NMS/Dermal topography/01-26-2026 mTmG/F2 DOB 12-5-2025/Left paw R1/G_align.tif is a low contrast image
  return func(*args, **kwargs)
100%|█████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 22.25it/s]


In [165]:
# Calculate curvatures

